### Bag of words / DTM(Document-Term Matrix)
- 단어의 순서를 고려하지 않고, 단어의 등장 빈도(frequency)만을 고려해 단어를 벡터화
    - 단점 : 문맥 무시, 오탈자, 줄임말 데이터로 갖고 있기 힘들다.
- <a href="https://youtu.be/dKYFfUtij_U" target="_blank"> Bag of words 유튜브 설명 </a>

[용어]
-  희소백터 : DTM의 문서 벡터나 단어 벡터는 대부분의 값이 0이라는 특징을 가지고 있는데, 이런 벡터를 칭함
-  단어장 : 중복 카운터 배제한 단어들의 집합(set), 단어의 중복을 허용하지 않는 집합

### TF-IDF
- 단어마다 중요 가중치를 다르게 주는 방법
- TF(Term Frequency)란 문장을 구성하는 단어들의 원-핫 벡터들을 모두 더해서 문장의 단어 갯수로 나눈 것과 같습니다.
    - TF 단점 : 불용어

## 벡터화 실습: 원-핫 인코딩 구현해보기
pip install konlpy

### Step 1. 패키지 설치하기

In [4]:
import re
from konlpy.tag import Okt
from collections import Counter
print("임포트 완료")

임포트 완료


In [5]:
text = "임금님 귀는 당나귀 귀! 임금님 귀는 당나귀 귀! 실컷~ 소리치고 나니 속이 확 뚫려 살 것 같았어."
text

'임금님 귀는 당나귀 귀! 임금님 귀는 당나귀 귀! 실컷~ 소리치고 나니 속이 확 뚫려 살 것 같았어.'

### Step 2. 전처리 이야기 

In [6]:
# 정규 표현식을 반영하여 전처리를 수행, 특수문자 제거

reg = re.compile("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]")
text = reg.sub('', text)
print(text)

임금님 귀는 당나귀 귀 임금님 귀는 당나귀 귀 실컷 소리치고 나니 속이 확 뚫려 살 것 같았어


- 한글, 공백을 제외한 모든 문자를 표현하는 regex : [^ㄱ-ㅎㅏ-ㅣ가-힣 ]

### Step 3. 토큰화 이야기

In [7]:
okt=Okt()
tokens = okt.morphs(text)
print(tokens)

['임금님', '귀', '는', '당나귀', '귀', '임금님', '귀', '는', '당나귀', '귀', '실컷', '소리', '치고', '나니', '속이', '확', '뚫려', '살', '것', '같았어']


### Step 4. 단어장 만들기
- <a href="https://docs.python.org/3/library/collections.html#collections.Counter" target="_blank"> 참고: 파이썬 collections.Counter </a>

In [9]:
vocab = Counter(tokens)
print(vocab)

Counter({'귀': 4, '임금님': 2, '는': 2, '당나귀': 2, '실컷': 1, '소리': 1, '치고': 1, '나니': 1, '속이': 1, '확': 1, '뚫려': 1, '살': 1, '것': 1, '같았어': 1})


In [10]:
#빈도수 출력해보기
vocab['임금님']


2

In [11]:
#  빈도수 상위 5개의 단어 확인
vocab_size=5
vocab=vocab.most_common(vocab_size) #등장 빈도수가 높은 상위 5개의 단어만 저장
print(vocab)

[('귀', 4), ('임금님', 2), ('는', 2), ('당나귀', 2), ('실컷', 1)]


In [12]:
# 높은 빈도수를 가진 단어에 낮은 정수 인덱스 부여

word2idx={word[0] : index+1 for index, word in enumerate(vocab)}
print(word2idx)

{'귀': 1, '임금님': 2, '는': 3, '당나귀': 4, '실컷': 5}


### Step 5: 원-핫 벡터 만들기

In [13]:
def one_hot_encoding(word, word2index):
       one_hot_vector = [0]*(len(word2index))
       index = word2index[word]
       one_hot_vector[index-1] = 1
       return one_hot_vector
print("슝=3")

슝=3


In [14]:
# 임금님 단어 원핫벡터 얻기

one_hot_encoding("임금님", word2idx)

[0, 1, 0, 0, 0]

### 케라스를 통한 원-핫 인코딩(one-hot encoding)

In [16]:
from tensorflow.keras.preprocessing.text import Tokenizer #단어장 만들기
from tensorflow.keras.utils import to_categorical #원-핫 인코딩 위한 도구
print("임포트 완료")

임포트 완료


In [19]:
# 위의 3개 문서 text 저장하기

text = [['강아지', '고양이', '강아지'],['애교', '고양이'], ['컴퓨터', '노트북']]
text

[['강아지', '고양이', '강아지'], ['애교', '고양이'], ['컴퓨터', '노트북']]

In [20]:
# 주어진 텍스트로 부터 단어장 만들고, 단어장의 각 단어에 고유한 정수 맵핑
t = Tokenizer()
t.fit_on_texts(text)
print(t.word_index) # 각 단어에 대한 인코딩 결과 출력.

{'강아지': 1, '고양이': 2, '애교': 3, '컴퓨터': 4, '노트북': 5}


In [21]:
# 단어장 크기 vocab_size라는 변수에 저장
vocab_size = len(t.word_index) + 1  #특별 토큰이 있어서 +1을 해준다

In [22]:
# 단어장에 속한 단어들로 구성된 텍스트 시퀀스는 케라스 토크나이저를 통해 정수 시퀀스로 변환
sub_text = ['강아지', '고양이', '강아지', '컴퓨터']
encoded = t.texts_to_sequences([sub_text])
print(encoded)

[[1, 2, 1, 4]]


In [23]:
# 변환된 정수 시퀀스를 to_categorical()을 사용해 원-핫 벡터의 시퀀스로 변환

one_hot = to_categorical(encoded, num_classes = vocab_size)
print(one_hot)

[[[0. 1. 0. 0. 0. 0.]
  [0. 0. 1. 0. 0. 0.]
  [0. 1. 0. 0. 0. 0.]
  [0. 0. 0. 0. 1. 0.]]]


### 워드 임베딩
- 차원의 저주 : 데이터에서 모델을 학습할 때 독립적 샘플이 많을수록 학습이 잘 되는 반면 차원이 커질 수록 학습이 어려워지고 더 많은 데이터를 필요로 합니다.
- <a href="http://thesciencelife.com/archives/1001" target="_blank"> 참고: 차원의 저주 </a>
- 단어간의 유사도를 구할수 없는 기계는 대안으로 워드 임베딩 제안, 각 차원이 다양한 실숫값을 가지며 이 밀집 벡터를 임베딩 벡터라고 한다. 

### word2vec
- <a href="https://youtu.be/sY4YyacSsLc" target="_blank"> 참고: word2vec </a>
- 분포 가설 :비슷한 문맥에서 같이 등장하는 경향이 있는 단어들은 비슷한 의미를 가진다.

### Word2Vec (2) CBoW
- 주변단어로 중심단어 예측 , 입력층, 은닉층, 출력층 이렇게 3개의 층으로만 구성된 인공 신경망이라는 의미
![cbow 시각화](https://user-images.githubusercontent.com/70866993/136575238-8dee5287-2c4e-497d-888f-ddc902e4ca11.png)


-  원-핫 벡터는 각 단어의 정수 인덱스 i에 해당되는 위치에만 1의 값을 가지므로, 원-핫 벡터와 가중치 행렬과의 곱은 가중치 행렬의 i 위치에 있는 행을 그대로 가져오는 것과 동일
    - <보조설명> 0,1중에 1만 살아 남기 때문에 1의 행만 슬라이딩 해서 가져온다.
- CBoW에서는 이 벡터들을 모두 합하거나, 평균을 구한 값을 최종 은닉층의 결과로 한다.
- Word2Vec에서는 은닉층에서 활성화 함수나 편향(bias)을 더하는 연산을 하지 않습니다.
    - 은닉층은 활성화 함수가 존재하지 않고 단순 가중치 행렬과 곱셈만을 수행하기에 투사층이라고도 한다.

- 은닉층에서 생성된 N차원의 벡터는 두 번째 가중치 행렬과 곱해집니다. 
- 이 가중치 행렬의 크기는 (N× V)이므로, 곱셈의 결과로 나오는 벡터의 차원은 V입니다. 
- 출력층은 활성화 함수로 소프트맥스 함수를 사용하므로 이 V차원의 벡터는 활성화 함수를 거쳐 모든 차원의 총합이 1이 되는 벡터로 변경됩니다.

### (3) Skip-gram
- Skip-gram : 중심 단어(input word)로부터 주변 단어(target word)를 예측, 그리고 이로 인해 '중간에 은닉층에서 다수의 벡터의 덧셈과 평균을 구하는 과정이 없어졌다점에서 차이가 있다.' 은닉층V에 AVG가 사라져있는것을 볼수 있음
![Skip-gram](https://user-images.githubusercontent.com/70866993/136576879-f401c133-1ed0-4d57-acac-2b341d3669ce.png)


### Negative Sampling
- 대체적으로 Word2Vec를 사용할 때는 SGNS(Skip-Gram with Negative Sampling)을 사용
- V개 중 1개를 고르는 다중 클래스 분류 문제를 시그모이드 함수를 사용한 이진 분류 문제로 바꾸기로 합니다.
- 랜덤으로 단어장에 있는 아무 단어나 가져와 target word로 하는 거짓 데이터셋을 만들고 0으로 레이블링을 해주는 것이죠! 거짓(negative) 데이터셋을 만들기 때문에 이 방법이 네거티브 샘플링입니다.

### 영어 Word2Vec 실습
-  Word2Vec을 별도로 구현할 필요없이 파이썬의 gensim 패키지를 통해 이미 구현된 Word2Vec 모델을 사용

In [25]:
5-7 30분, 20, 30, 45분

SyntaxError: invalid syntax (<ipython-input-25-8165c786d45d>, line 1)

- pip install nltk
- pip install gensim

In [26]:
import nltk
nltk.download('abc')
nltk.download('punkt')

[nltk_data] Downloading package abc to /aiffel/nltk_data...
[nltk_data]   Package abc is already up-to-date!
[nltk_data] Downloading package punkt to /aiffel/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [27]:
from nltk.corpus import abc
corpus = abc.sents()
print("슝~")

슝~


In [28]:
from nltk.corpus import abc
corpus = abc.sents()
print("슝~")

슝~


In [29]:
print('코퍼스의 크기 :',len(corpus))

코퍼스의 크기 : 29059


In [30]:
from gensim.models import Word2Vec

model = Word2Vec(sentences = corpus, vector_size = 100, window = 5, min_count = 5, workers = 4, sg = 0)
print("모델 학습 완료!")

/opt/conda/lib/python3.7/site-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


모델 학습 완료!


- vector size = 학습 후 임베딩 벡터의 차원
- window = 컨텍스트 윈도우 크기
- min_count = 단어 최소 빈도수 제한 (빈도가 적은 단어들은 학습하지 않아요.)
- workers = 학습을 위한 프로세스 수
- sg = 0은 CBoW, 1은 Skip-gram.

In [31]:
model_result = model.wv.most_similar("man")
print(model_result)

[('woman', 0.9233418107032776), ('skull', 0.911030113697052), ('Bang', 0.905648946762085), ('asteroid', 0.9052114486694336), ('third', 0.9020071625709534), ('baby', 0.8994219303131104), ('dog', 0.898607611656189), ('bought', 0.8975202441215515), ('rally', 0.8912495374679565), ('disc', 0.8889137506484985)]


In [32]:
from gensim.models import KeyedVectors

model.wv.save_word2vec_format('~/aiffel/word_embedding/w2v') 
loaded_model = KeyedVectors.load_word2vec_format("~/aiffel/word_embedding/w2v")
print("모델  load 완료!")

모델  load 완료!


In [33]:
model_result = loaded_model.most_similar("man")
print(model_result)

[('woman', 0.9233418107032776), ('skull', 0.911030113697052), ('Bang', 0.905648946762085), ('asteroid', 0.9052114486694336), ('third', 0.9020071625709534), ('baby', 0.8994219303131104), ('dog', 0.898607611656189), ('bought', 0.8975202441215515), ('rally', 0.8912495374679565), ('disc', 0.8889137506484985)]


### Word2Vec의 OOV 문제  *단어장에 없어서 오류남

In [34]:
# 에러가 나더라도 놀라지 마세요.
loaded_model.most_similar('overacting')

KeyError: "Key 'overacting' not present"

In [ ]:
loaded_model.most_similar('memorry')

### 임베딩 벡터의 시각화
- https://projector.tensorflow.org/ 군집 확인
- 구글이 공개한 임베디 벡터의 시각화 오픈소스인 임베딩 프로젝터(embedding projector)를 사용해서 임베딩 벡터들을 시각화합니다.
- 임베딩 프로젝터를 통해 어떤 임베딩 벡터들이 가까운 거리에 군집이 되어있고, 특정 임베딩 벡터와 유클리드 거리나 코사인 유사도가 높은지 확인할 수 있습니다.
- 임베딩 프로젝터를 통해 임베딩 벡터를 시각화하기 위해서는 이미 저장된 모델이 필요합니다.
- 이미 저장된 모델로부터 벡터값이 저장된 파일과 메타파일을 얻어야하기 때문입니다.
- 임베딩 프로젝터 사이트는 좌측 상단을 통해 데이터를 업로드하고, 시각화 결과를 중앙에서 볼 수 있으며, 우측에서 거리나 유사도에 대한 파라미터를 조작할 수 있는 구조입니다.
- 우측에 Search 버튼 또는 그래프의 포인트를 클릭해 원하는 단어를 선택하고, neighbors에 몇 개까지의 이웃을 검색할지 선택합니다.
- distance에서 COSINE 또는 EUCLIDEAN을 통해서 거리 측정 메트릭을 코사인 유사도로 할 것인지, 유클리드 거리로 할 것인지 선택할 수 있습니다.
- 참고: <a href="https://radimrehurek.com/gensim/models/word2vec.html" target="_balnk"> gensim scripts,word2vec2tensor</a> 
<a href="https://projector.tensorflow.org/" target="_balnk"> Embedding Projector.</a>

In [ ]:
#- w2v_metadata.tsv와 w2v_tensor.tsv 파일 생성
! python -m gensim.scripts.word2vec2tensor --input ~/aiffel/GoingDeeper/DATA/word_embedding/w2v --output ~/aiffel/GoingDeeper/DATA/word_embedding/w2v

### FastText

- 페이스북에서 개발한 FastText는 Word2Vec이후에 등장한 워드 임베딩 방법입니다.
- 메커니즘 자체는 Word2Vec을 그대로 따르지만, 문자 단위 n-gram(chacracter-level n-gram)표현을 학습한다는 점에서 다릅니다.
- Word2Vec은 단어를 더 이상 깨질 수 없는 단위로 구분하는 반면, FastText는 단어 내부의 내부단어(subwords)들을 학습한다는 아이디어를 가지고 있습니다.
- FastText의 n-gram에서 n은 단어들이 얼마나 분리되는지 결정하는 하이퍼파라미터입니다.
- n을 3으로 잡은 트라이그램(tri-gram)의 경우, 단어 "partial"은 'par','art','rti','tia','ial'로 분리하고 이들을 벡터로 만듭니다.
- 더 정확히는 시작과 끝을 의미하는 \<,>를 도입하여 \<pa, are, rti, tia, ial, al> 라는 6개의 내부 단어(subword) 토큰을 벡터로 만듭니다. 여기에 추가로 기존 단어에 \<,> 를 붙인 토큰 \<partial>를 벡터화합니다.
- 즉 n=3인 경우, FastText는 단어 partial 에 대해 임베딩되는 n-gram 토큰은 다음과 같습니다.
    - \<pa, art, rti, tia, ial, al>, \<partial>
- 실제 사용시에는 n의 최솟값, 최댓값으로 범위를 설정할 수 있는데, gensim 패키지에서는 기본값으로 각각 3과 6으로 설정되어 있습니다.
    - \<pa, art, rti, ita, ial, al>, \<par, arti, rtia, tial, ial>, \<part, ...중략... , \<partial>
- 내부단어들을 벡터화한다는 의미는 위 단어들 각각에 대해서 Word2Vec을 수행한다는 의미입니다. 최종적으로 벡터화된 n-gram 벡터들의 총합을 해당 단어의 벡터로 취합니다.

    - partial = \<pa + art + rti + ita + ial + a\l> + \<par + arti + rtia + tial + ia\l> + \<part + ...중략... + \<partial>

In [ ]:
from gensim.models import FastText
fasttext_model = FastText(corpus, window=5, min_count=5, workers=4, sg=1)
print("FastText 학습 완료!")

In [ ]:
fasttext_model.wv.most_similar('overacting')

In [ ]:
fasttext_model.wv.most_similar('memoryy')

#### FastText의 학습방법
- FastText도 Word2Vec과 마찬가지로 네거티브 샘플링을 사용하여 학습합니다.
- "(중심 단어, 주변 단어)"의 쌍을 가지고 이 쌍이 포지티브인지 네거티브인지 예측을 진행하는 것입니다.
- 다만, Word2Vec와 다른 점은 학습 과정에서 중심 단어에 속한 문자 단위 n-gram 단어 벡터들을 모두 업데이트한다는 점입니다.

#### OOV 와 오타에 대한 대응
- FastText는 Word2Vec과 달리 OOV와 오타에 robust(강건한)한 특징이 있습니다.
- 이는 단어장에 없는 단어라도, 해당 단어의 n-gram이 다른 단어에 존재하면 이로부터 벡터값을 얻는다는 원리에 기인합니다.

#### 한국어에서의 FastText
- 영어의 경우 문자, 즉 알파벳 단위가 n-gram이었다면 한국어의 경우는 음절 단위라고 볼 수 있습니다.
- 한국어 어휘를 자모 수준으로 분리하여 학습하는 것은 일부 어휘의 특성을 학습하는 데 큰 도움이 되지 않거나 가끔은 성능의 저하를 일으키기 때문에, 아보카도와 같은 고유 명사는 분해해도 이득을 볼 수 없습니다.
- 음절 단위 FastText
    - n=3일때 단어 '아보카도씨앗'의 트라이그램 벡터
    - \<아보, 아보카, 보카도, 카도씨, 도씨앗, 씨앗>
    - 한국어에서 FastText가 빛을 발하는 것은 음절 단위보다는 자소 단위입니다.
    - 한국어의 자소를 각각의 문자로 간주한 경우, FastText는 꽤 잘 동작한다고 알려져있습니다.
- 자소 단위 FastText
    - 단어에 대해서 초성, 중성, 종성을 분리한다고 하고, 종성이 존재하지 않는 경우에 _ 라는 토큰을 사용
        - \<ㅇㅏ,ㅇㅏ_,ㅏ_ㅂ,중략..>
    - 참고 :<a href="https://brunch.co.kr/@learning/8" target="_balnk"> 한국어 어휘임베딩</a> 

###  GloVe
- 글로브(Gloval Vectors for Word Representation, GloVe)는 2014년에 미국 스탠포드 대학에서 개발한 워드 임베딩 방법론입니다.

- 워드 임베딩의 두 가지 접근 방법인 카운트 기반과 예측 기반 두 가지 방법을 모두 사용했다는 것이 특징입니다.

- 행렬 A에 대해서 특이값 분해의 변형인 thin SVD, compact SVD, Truncated SVD를 각각 사용하였을 때 thin SVD, compact SVD 두 가지는 행렬 A를 복원할 수 있습니다.
- 잠재 의미 분석(LSA, Latent Semantic Analysis)은 DTM에 특잇값 분해를 사용하여 잠재된 의미를 이끌어내는 방법론입니다.
- LSA는 단어를 카운트해서 만든 DTM을 입력으로 하므로 카운트 기반의 임베딩 방법이라고 볼 수 있는데, 이 방법은 몇 가지 한계가 있었습니다.
- 차원 축소의 특성으로 인해 새로운 단어가 추가되면 다시 DTM을 만들어 새로 차원축소를 해야 합니다.
- 단어 벡터간 유사도를 계산하는 측면에서 Word2Vec보다 성능이 떨어집니다.
- 반면 LSA와 대조되는 방법으로 예측 기반의 방법은 Word2Vec과 같은 방법입니다.
- Word2Vec은 인공신경망이 에측한 값으로부터 실제 레이블과의 오차를 구하고, 손실함수를 통해서 인공 신경망을 학습하는 방법입니다.
- GloVe 연구진은 Word2Vec의 경우에는 LSA보다 단어 벡터 간 유사도를 구하는 능력은 뛰어나지만, LSA처럼 코퍼스의 전체적인 통계 정보를 활용하지 못한다는 점을 한계로 지적했습니다.
 그리고 카운트 기반과 예측 기반을 모두 사용하여 Word2Vec보다 더 나은 임베딩 방법을 GloVe를 제안합니다.

#### -윈도우 기반 동시 등장 행렬(Window based Co-occurrence Matrix)
- 윈도우 기반 동시 등장 행렬은 행과 열을 전체 단어장(vocabulary)의 단어들로 구성하고, 어떤 i 단어의 윈도우 크기(window Size)내에서 k 단어가 등장한 횟수를 i행 k열에 기재한 행렬입니다.
- 동시 등장 행렬은 전치(transpose)해도 동일한 행렬이 된다는 특징을 가지고 있습니다.

#### 동시 등장 확률(Co-occurrence Probability)
- 동시 등장 확률 P(k∣i)는 동시 등장 행렬로부터 특정 단어 i의 전체 등장 횟수를 카운트하고, 특정 단어 i가 등장했을 때 어떤 단어 k가 등장한 횟수를 카운트하여 계산한 조건부 확률입니다.
- 이때 i를 중심 단어(center word), k를 주변 단어(context word)입니다.

#### GloVe의 손실 함수 설계하기
- GloVe는 동시 등장 행렬로부터 계산된 동시 등장 확률을 이용해 손실 함수를 설계합니다.
- 동시 등장 행렬을 사용하고 있으니 코퍼스의 전체적인 통계 정보를 활용하는 '카운트 기반'의 방법론이면서, 손실 함수를 통해 모델을 학습시키므로 '예측 기반'의 방법론이라고 할 수 있는 것입니다.
- GloVe는 중심 단어 벡터와 주변 단어 벡터의 내적이 전체 코퍼스에서의 동시 등장 빈도의 로그값이 되도록 만든다는 아이디어입니다.
다시말하면, 전체 코퍼스에서의 동시 등장 빈도의 로그값과 중심 단어 벡터와 주변 단어 벡터의 내적값의 차이가 최소화되도록 두 벡터의 값을 학습하는 것입니다.
- 참고 : <a href="https://ratsgo.github.io/from%20frequency%20to%20semantics/2017/04/06/pcasvdlsa/" target="_blank"> SVD,PCA,LSA</a>  <a href="https://www.youtube.com/watch?v=GVPTGq53H5I" target="_blank"> LSA 설명영상</a> <a href="http://web.stanford.edu/class/cs224n/slides/cs224n-2019-lecture02-wordvecs2.pdf" target="_blank"> 동시등장행렬 </a> <a href="https://wikidocs.net/22885" target="_blank"> 글로브 </a>

In [2]:
# ! conda create -n glove python=3.6
# ! pip install glove_python_binary
# ! pip install nltk

In [3]:
import nltk
nltk.download('movie_reviews')
nltk.download('punkt')

[nltk_data] Downloading package movie_reviews to /aiffel/nltk_data...
[nltk_data]   Unzipping corpora/movie_reviews.zip.
[nltk_data] Downloading package punkt to /aiffel/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [4]:
from nltk.corpus import movie_reviews
corpus=movie_reviews.sents()

In [5]:
from glove import Corpus, Glove

# 훈련 데이터로부터 GloVe에서 사용할 동시 등장 행렬 생성
emb = Corpus() 
emb.fit(corpus, window=5)

# 벡터의 차원은 100, 학습에 이용할 쓰레드의 개수는 4로 설정, 에포크는 20.
glove = Glove(no_components=100, learning_rate=0.05)
glove.fit(emb.matrix, epochs=20, no_threads=4, verbose=True)
glove.add_dictionary(emb.dictionary)

ModuleNotFoundError: No module named 'glove'

In [6]:
model_result1 = glove.most_similar("man")
model_result2 = glove.most_similar("fiction")

print("model_result1", model_result1)
print("model_result2", model_result2)

NameError: name 'glove' is not defined